In [2]:
import pandas as pd
import numpy as np
import mysql.connector

In [3]:
# Set up a MySQL connection
cnx = mysql.connector.connect(user='root', password='my123sql',
                              host='localhost', database='intest')

In [4]:
class dataframes:
    user=test=questions=options=responses=olap_test=olap_question=pd.DataFrame()
    def __init__(self):
        cnx = mysql.connector.connect(user='root', password='my123sql',
                              host='localhost', database='intest')
        
        query_user = "SELECT * FROM users"
        self.user = pd.read_sql(query_user, cnx)
        
        query_test = "SELECT * FROM test_db"
        self.test= pd.read_sql(query_test, cnx)
        
        query_questions = "SELECT * FROM questions"
        self.questions= pd.read_sql(query_questions, cnx)
        
        query_options = "SELECT * FROM mcq_options"
        self.options= pd.read_sql(query_options, cnx)
        
        query_responses = "SELECT * FROM responses"
        self.responses= pd.read_sql(query_responses, cnx)
        
        query_olap_test = "SELECT * FROM olap_test"
        self.olap_test= pd.read_sql(query_olap_test, cnx)
        
        query_olap_question = "SELECT * FROM olap_question"
        self.olap_question= pd.read_sql(query_olap_question, cnx)
        
        cnx.close()

In [5]:
class OLAP_Test:
    df:dataframes
    results={'id': 0, 'testId':0, 'totalAppeared':0, 'maxMarks':0, 'highestMarks':0, 'lowestMarks':0,
             'avgMarks':0, 'noOfPassed':0, 'noOfFailed':0, 'lastUpdated': ''}
    testId=0
    
    def __init__(self, df: dataframes, testId: int) -> None:
        self.df=df
        self.testId=testId
        self.results['testId']=testId
        
        
    def calculate(self):
        
        responses=self.df.responses.query('testId==@self.testId')
        test=self.df.test.query('id==@self.testId')
        
        def setTotalAppeared():
            self.results['totalAppeared']=len(responses['userId'].unique())
        
        def setMaxMarks():
            mm=test['maxMarks'].loc[0]
            self.results['maxMarks']=mm
        
        def setHighestMarks():
            max_score = responses.groupby('userId')['obtainedMarks'].sum().max()
            self.results['highestMarks']=max_score
            
        def setLowestMarks():
            min_score = responses.groupby('userId')['obtainedMarks'].sum().min()
            self.results['lowestMarks']=min_score
        
        def setAvgMarks():
            average_marks = responses.groupby('userId')['obtainedMarks'].sum().mean()
            self.results['avgMarks']=average_marks
        
        def setNoOfPassedandFailed():
            passMarks=test.passMarks.loc[0]
            passed=(responses.groupby('userId')['obtainedMarks'].sum() >= passMarks).sum()
            self.results['noOfPassed']=passed
            self.results['noOfFailed']=self.results['totalAppeared']-passed
        setTotalAppeared()
        setMaxMarks()
        setHighestMarks()
        setLowestMarks()
        setAvgMarks()
        setNoOfPassedandFailed()
        return self.results
 

In [6]:
#analyzes one question by its ID
class OLAP_questions:
    df:dataframes
    result={'id': 0, 'questionId':0, 'maxMarks':0, 'highestMarks':0, 'lowestMarks':0,
             'avgMarks':0, 'topic': ''}
    testId:int
    
    #maps dataframes created and locks the testId
    def __init__(self, df:dataframes, testId: int):
        self.df=df
        self.testId=testId
        
    #calculates analysis for a particular question by questionId
    def calculate(self,questionId:int):
        responses=self.df.responses.query('questionId==@questionId')
        question=self.df.questions.query('id==@questionId')
        self.result['questionId']=questionId
        self.result['topic']=question['topic'].iloc[0]
        self.result['maxMarks']=question['maxMarks'].iloc[0]
        self.result['highestMarks']=responses['obtainedMarks'].max()
        self.result['lowestMarks']=responses['obtainedMarks'].min()
        self.result['avgMarks']=responses['obtainedMarks'].mean()

        return self.result       

In [20]:

#analyzes test scores wrt topics
class studentAnalysis:
    testId:int
    df:dataframes
    def __init__(self,df:dataframes) -> None:
        self.df=df
        
    def getTestData(self, testId:int, studentId: int):
        olapTest=OLAP_Test(df=self.df,testId=testId)
        results=olapTest.calculate()
        responses=self.df.responses.query('testId==@testId & userId==@studentId')
        results['studentId']=studentId
        results['totalMarksObtained']=responses['obtainedMarks'].sum()
        return results
    
    def getQuestionData(self, testId:int, studentId: int, questionId: int):
        olapq=OLAP_questions(df=self.df,testId=testId)
        result=olapq.calculate(questionId=questionId)
        response=self.df.responses.query('testId==@testId & userId==@studentId & questionId==@questionId')
        result['obtainedMarks']=response['obtainedMarks'].iloc[0]
        return result
    
    def getStudents(self,testId:int):
        responses=self.df.responses.query('testId==@testId')
        userIdList=responses.userId.unique()
        results=[]
        for id in userIdList:
            data={}
            user=self.df.user.query('id==@id').iloc[0]
            data['userId']=id
            data['firstName']=user['firstName']
            data['lastName']=user['lastName']
            data['obtainedMarks']=self.getTestData(testId=testId,studentId=id)['totalMarksObtained']
            results.append(data)
        return results
    
    def getTopicAnalysis(self, testId:int, userId:int):
        questions = self.df.questions.query('testId==@testId')
        responses = self.df.responses.query('testId==@testId')
        topics=questions['topic'].unique()
        
        def data(topic:str,averageMarks:float,yourMarks:int,highestMarks:int,lowestMarks:int):
            body={}
            body['topic']=topic
            body['averageMarks']=averageMarks
            body['yourMarks']=yourMarks
            body['highestMarks']=highestMarks
            body['lowestMarks']=lowestMarks
            return body

        merged_df = questions.merge(responses, left_on='id', right_on='questionId')
        results=[]
        
        merged_df = pd.merge(questions, responses, left_on='id', right_on='questionId')

        user_marks_sum = merged_df.groupby(['topic', 'userId'])['obtainedMarks'].agg('sum').reset_index()
        most_marks_by_user = user_marks_sum.groupby('topic').agg('max').reset_index()
        avg_marks_by_topic = merged_df.groupby('topic')['obtainedMarks'].agg('mean').reset_index()
        least_marks_by_user = user_marks_sum.groupby('topic').agg('min').reset_index()
        
        for topic in topics:
            yourMarks=user_marks_sum.query('topic==@topic & userId==@userId')['obtainedMarks'].iloc[0]
            highestMarks=most_marks_by_user.query('topic==@topic')['obtainedMarks'].iloc[0]
            averageMarks=avg_marks_by_topic.query('topic==@topic')['obtainedMarks'].iloc[0]
            lowestMarks=least_marks_by_user.query('topic==@topic')['obtainedMarks'].iloc[0]
            results.append(data(topic=topic,averageMarks=averageMarks,yourMarks=yourMarks,highestMarks=highestMarks,lowestMarks=lowestMarks))
        return results
                    
            

In [21]:
df=dataframes()
sa=studentAnalysis(df)
print(sa.getTestData(testId=1, studentId= 2))
print(sa.getQuestionData(testId=1,studentId=3,questionId=1))
print(sa.getStudents(1))

{'id': 0, 'testId': 1, 'totalAppeared': 4, 'maxMarks': 15, 'highestMarks': 15, 'lowestMarks': 0, 'avgMarks': 7.5, 'noOfPassed': 2, 'noOfFailed': 2, 'lastUpdated': '', 'studentId': 2, 'totalMarksObtained': 15}
{'id': 0, 'questionId': 1, 'maxMarks': 5, 'highestMarks': 5, 'lowestMarks': 0, 'avgMarks': 2.5, 'topic': 'Cities', 'obtainedMarks': 0}
[{'userId': 2, 'firstName': 'Shahrukh', 'lastName': 'Khan', 'obtainedMarks': 15}, {'userId': 3, 'firstName': 'Salman', 'lastName': 'Khan', 'obtainedMarks': 0}, {'userId': 4, 'firstName': 'Aamir', 'lastName': 'Khan', 'obtainedMarks': 5}, {'userId': 5, 'firstName': 'Hrithik', 'lastName': 'Roshan', 'obtainedMarks': 10}]


In [9]:
questions=df.questions.query('testId==1')
topics=questions['topic'].unique()
topics

array(['Cities', 'Government', 'Animals'], dtype=object)

In [16]:
questions = df.questions
responses = df.responses

merged_df = pd.merge(questions, responses, left_on='id', right_on='questionId')

user_marks_sum = merged_df.groupby(['topic', 'userId'])['obtainedMarks'].agg('sum').reset_index()
most_marks_by_user = user_marks_sum.groupby('topic').agg('max').reset_index()
avg_marks_by_topic = merged_df.groupby('topic')['obtainedMarks'].agg('mean').reset_index()
least_marks_by_user = user_marks_sum.groupby('topic').agg('min').reset_index()


# display(merged_df)
# display(user_marks_sum)
# display(most_marks_by_user)
# display(avg_marks_by_topic)
# display(least_marks_by_user)

most_marks_by_user.query('topic=="Animals"')['obtainedMarks'].iloc[0]

5

In [22]:
ans=sa.getTopicAnalysis(testId=1,userId=4)
ans

[{'topic': 'Cities',
  'averageMarks': 2.5,
  'yourMarks': 0,
  'highestMarks': 5,
  'lowestMarks': 0},
 {'topic': 'Government',
  'averageMarks': 2.5,
  'yourMarks': 5,
  'highestMarks': 5,
  'lowestMarks': 0},
 {'topic': 'Animals',
  'averageMarks': 2.5,
  'yourMarks': 0,
  'highestMarks': 5,
  'lowestMarks': 0}]